# 얼굴 분류기 
### _ 무한도전 중 승리

### GitHub 모델 이용 https://github.com/davisking/dlib-models 

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import dlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import tensorflow.keras 
from tensorflow.keras import backend as K

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat')

In [11]:
def find_faces(img):
    dets = detector(img, 1)
    
    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        # convert dlib shape to numpy array
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np

def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)

In [38]:
img_paths = {
     #'진호': 'user_img/jinho.jpg',
     '승리one': 'del_img/victory2.jpg',
     '승리two': 'del_img/victory1.jpg',
}

descs = []

for name, img_path in img_paths.items(): 
    img = cv2.imread(img_path)
    _, img_shapes, _ = find_faces(img)
    descs.append([name, encode_faces(img, img_shapes)[0]])
    
np.save('user_img/descs.npy', descs)
print(descs)

[['승리one', array([-2.40262132e-02,  9.39716250e-02,  4.13361937e-02, -4.57958393e-02,
       -1.03981122e-01, -3.15149613e-02, -5.27299419e-02, -1.32668108e-01,
        1.32559806e-01, -8.61146748e-02,  2.70637333e-01,  7.65107339e-03,
       -2.36778975e-01, -6.70214444e-02, -8.10003504e-02,  1.85054198e-01,
       -1.62715316e-01, -1.71724036e-01, -6.58934563e-02,  3.38285193e-02,
        1.11694440e-01,  5.01411073e-02,  2.06927564e-02,  7.17308233e-03,
       -1.07667871e-01, -2.44589716e-01, -6.71811923e-02, -5.05602285e-02,
       -1.96790062e-02, -6.58920035e-02, -8.24876130e-02,  9.94025962e-04,
       -1.97627902e-01, -8.78196806e-02,  1.05078720e-01,  6.55246750e-02,
       -4.50344831e-02, -1.36058539e-01,  2.01559290e-01,  4.80270572e-03,
       -2.19820619e-01,  8.11564624e-02,  6.56966716e-02,  2.26459309e-01,
        2.10406929e-01, -2.25828625e-02,  1.42756533e-02, -2.19403371e-01,
        1.54290468e-01, -1.85518995e-01,  5.92763983e-02,  1.17612146e-01,
        1.3617

In [39]:
cam = cv2.VideoCapture("Images/기피대상.mp4") # 동영상
cam.set(3, 640) # 너비
cam.set(4, 480) # 높이
font = cv2.FONT_HERSHEY_SIMPLEX # 폰트

while True:
    
    ret, img = cam.read() 
    rects, shapes, _ = find_faces(img) # 얼굴 찾기
    descriptors = encode_faces(img, shapes) # 인코딩
    
    for i, desc in enumerate(descriptors):
        x = rects[i][0][0] # 얼굴 X 좌표
        y = rects[i][0][1] # 얼굴 Y 좌표
        w = rects[i][1][1]-rects[i][0][1] # 얼굴 너비 
        h = rects[i][1][0]-rects[i][0][0] # 얼굴 높이
        
        # 추출된 랜드마크와 데이터베이스의 랜드마크들 중 제일 짧은 거리를 찾는 부분
        descs1 = sorted(descs, key=lambda x: np.linalg.norm([desc] - x[1]))
        dist = np.linalg.norm([desc] - descs1[0][1], axis=1)
        
        if dist < 0.45: # 그 거리가 0.45보다 작다면 그 사람으로 판단 
            mosaic_img = cv2.resize(img[y:y+h, x:x+w], dsize=(0, 0), fx=0.04, fy=0.04) # 축소
            mosaic_img = cv2.resize(mosaic_img, (w, h), interpolation=cv2.INTER_AREA)  # 확대
            img[y:y+h, x:x+w] = mosaic_img

#     한글
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    img = np.array(img)
        
    cv2.imshow('camera', img)
    
    k = cv2.waitKey(10) & 0xff # 'ESC' 키 누르면 종료
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()